In [ ]:
import os, glob
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
import scipy.stats
import neural_helpers as nh

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## GLMSingle

In [13]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
from glmsingle.glmsingle import GLM_single

BASEDIR='/Volumes/shohamy-locker/chris/hybrid_mri_CSI' # /Volumes/shohamy-locker/shohamy_from_labshare/rgerraty/hybrid_mri
behavior_dir='/Volumes/shohamy-locker/shohamy_from_labshare/rgerraty/hybrid_mri/behavior'
bold_ext = 'fmriprep_bold_smoothed_masked.nii.gz' # 'preproc_6mm_6del_100s_mc/filtered_func_data.nii.gz
# confounds_ext = 'fmriprep_confounds_24par.txt' # preproc_6mm_6del_100s_mc/mc/extended_confs_24par.txt
OUTDIR='~/Downloads'

TR=2
STIMDUR=1.5 # modeling both the choice (response time + 3s delay) and FB (1.5s) periods as 1.5s

In [ ]:
# for sub_id in sorted([i for i in os.listdir(BASEDIR) if 'TCST' in i]):
#     sub_num=sub_id[-2:]
#     subdir = os.path.join(BASEDIR, sub_id)
#     for run_id in sorted([j for j in os.listdir(subdir) if 'hybrid' in j]):
#         run_num=run_id[-1:]
sub_id = 'TCST002'
sub_num=sub_id[-2:]
subdir = os.path.join(BASEDIR, sub_id)
run_id = f'hybrid_r1'
run_num=run_id[-1:]

# load bold file
bold_path = os.path.join(subdir, run_id, bold_ext)
bold_data = nib.load(bold_path).get_fdata()

# create design matrix from behavior files
choice_ev = os.path.join(behavior_dir, f'{sub_num}_output/EV_files/choice_run{run_num}.txt')
fb_ev = os.path.join(behavior_dir, f'{sub_num}_output/EV_files/FB_run{run_num}.txt')


######################################
# Prepare design matrix (GLMsingle expects onsets, durations, conditions)
onsets = events['onset'].values
durations = events['duration'].values
conditions = events['trial_type'].values

# Build the design matrix
unique_conditions = np.unique(conditions)
design = np.zeros((bold_data.shape[-1], len(unique_conditions)))

for cond_idx, cond in enumerate(unique_conditions):
    trial_onsets = onsets[conditions == cond]
    design[trial_onsets.astype(int), cond_idx] = 1

# Run GLMsingle
glmsingle_params = dict()
glmsingle_params['wantlibrary'] = 1
glmsingle_params['wantglmdenoise'] = 1
glmsingle_params['wantfracridge'] = 1
glmsingle_params['wantfileoutputs'] = [0,0,0,1] # only final outputs
glmsingle_params['wantmemoryoutputs'] = [0,0,0,0]

glm = GLM_single(glmsingle_params)
results_glmsingle = glm.fit(design, bold_data, savedir)

print("GLMsingle complete! Results saved to:", savedir)


# Hippocampal ROI

# Pattern analyses

In [ ]:
beh = pd.read_csv('./data/hybrid_data.csv')